In [1]:
from IPython.core.pylabtools import figsize
figsize(12, 6)

import imageio
import matplotlib.pyplot as plt
import numpy as np
from utoolbox.data  import SPIMDataset



In [ ]:
from utils import find_dataset_dir, data_dirs

dataset_name = 'predict'
path = find_dataset_dir(dataset_name)
print(path)
dataset = SPIMDataset(path)

dataset_name_decon = 'predict_deconv'
path_decon = find_dataset_dir(dataset_name_decon)
print(path_decon)
dataset_decon = SPIMDataset(path_decon)

unable to parse the timestamp
unknown section ".ini File", ignored


G:/Vins/Mito_Binarization/11292017/c6_zp3_7ms_1s\predict


In [1]:
print(list(dataset.keys()))
print(list(dataset['0'].keys())[:3])
image = dataset['0']['c6_ch0_stack0322_488nm_0319851msec_0006234971msecAbs']
plt.subplot(121)
plt.imshow(image.max(axis=0))

print(list(dataset_decon.keys()))
print(list(dataset_decon['0'].keys())[:3])
image_decon = dataset_decon['0']['c6_ch0_stack0322_488nm_0319851msec_0006234971msecAbs_decon']
plt.subplot(122)
plt.imshow(image_decon.max(axis=0))



NameError: name 'dataset' is not defined

In [ ]:
from skimage.transform import resize
img_resize = resize(image_decon, (140,512,512))
print(img_resize.dtype)

from skimage.filters import median
from skimage.morphology import cube

mask = cube(3)
img_med = image.copy()
for i in range(10):
    print(f'iter {i}')
    img_med = median(img_med, mask)

means = []
for layer in img_med:
    means.append(layer.mean())
means = np.array(means)

mean, median = means.mean(), np.median(means)
print(f'mean: {mean:.1f}, median: {median:.1f}')



In [ ]:
from skimage.filters import apply_hysteresis_threshold

img_hyst = apply_hysteresis_threshold(img_med, mean, median)
plt.imshow(img_hyst[36, ...])
print(img_hyst.shape)



In [ ]:
from scipy.ndimage.morphology import binary_fill_holes

img_filled = binary_fill_holes(img_hyst)
plt.imshow(img_filled[36, ...])

imageio.volwrite('img_filled.tif', img_filled.astype(np.uint8))

In [ ]:
from scipy.ndimage.measurements import label

img_label, n_features = label(img_filled)
print(n_features)
print(f'img_filled: {img_filled.dtype}')
print(f'label_dtype: {img_label.dtype}')
plt.subplot(121)
plt.imshow(img_filled[36, ...])
plt.subplot(122)
plt.imshow(img_label[36, ...])

volumes = [(index, (img_label == index).sum()) for index in range(n_features)]
volumes.sort(key=lambda x: x[1])
print(volumes)

# 1st: background
threshold = volumes[-2][1] // 2
print(f'threshold: {threshold}')

img_kept = np.zeros_like(img_label)
# keep objects above threshold
for index, volume in volumes[:-1]:
    if volume > threshold:
        img_kept[img_label == index] = 1
img_kept = img_kept.astype(np.bool)

imageio.volwrite('img_kept.tif', img_kept.astype(np.uint8))
print(img_kept.dtype)
plt.imshow(img_kept[36, ...])